# Lab 4 Project (Titanic)
Jason Ballard
4 April 2025

Import the external Python libraries used (e.g., pandas, numpy, matplotlib, seaborn, sklearn and more).

## Section 1. Import and Inspect the Data

In [1]:
# all imports get moved to the top - import each only once

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# Load Titanic dataset
df = sns.load_dataset('titanic')

features = list(df.columns)
print(features)
print(len(features))

['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town', 'alive', 'alone']
15


In [3]:
# titanic.info()
# print(titanic.head(10))
# titanic.isnull().sum()
# print(titanic.describe())
# print(titanic.corr(numeric_only=True))

# Section 2. Data Exploration and Preparation

In [4]:
# attributes = ['age', 'fare', 'pclass']
# scatter_matrix(titanic[attributes], figsize=(10, 10))

In [5]:
# plt.scatter(titanic['age'], titanic['fare'], c=titanic['sex'].apply(lambda x: 0 if x == 'male' else 1))
# plt.xlabel('Age')
# plt.ylabel('Fare')
# plt.title('Age vs Fare by Gender')
# plt.show()

In [6]:
# #Create a histogram of age:

# sns.histplot(titanic['age'], kde=True)
# plt.title('Age Distribution')
# plt.show()

In [7]:
# #Create a count plot for class and survival:

# sns.countplot(x='class', hue='survived', data=titanic)
# plt.title('Class Distribution by Survival')
# plt.show()

In [8]:
df['age'].fillna(df['age'].median())

titanic = df.dropna(subset=['fare'])

titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1

<!-- ### Reflection 2.1:

1. What patterns or anomalies do you notice? Young to middle age passengers, majority found in third class
2. Do any features stand out as potential predictors? the deck location or fare price
3. Are there any visible class imbalances? There are huge class imbalances. Majority of the passengers where younger families traveling to the USA -->

## 2.1 Handle Missing Values and Clean Data

In [12]:
# Impute missing values in 'age' with the median age
df['age'].fillna(df['age'].median())

# Impute missing values in 'embark_town' with the mode (most common value)
mode_val = df['embark_town'].mode()
if not mode_val.empty:
    df['embark_town'].fillna(mode_val[0])
else:
    print("No mode found for 'embark_town'")


No mode found for 'embark_town'


## 2.2 Feature Engineering

In [10]:
# Create a new feature: Family size
# This is the coveresion of catagorical dat inot numerical data

df['family_size'] = df['sibsp'] + df['parch'] + 1 
df['sex'] = df['sex'].map({'male': 0, 'female': 1})
df['embark_town'] = df['embark_town'].map({'C': 0, 'Q': 1, 'S': 2})
df['alone'] = df['alone'].astype(int)

<!-- ### Reflection 2.3

1. Why might family size be a useful feature for predicting survival? famil;y size is a good prediction of survivalbility for the female and younger children of the families
2. Why convert categorical data to numeric?  the conversion allows computations to be run on the data. -->

# Section 3. Feature Selection and Justification

- Select two or more input features (numerical for regression, numerical and/or categorical for classification)
- Use 'Survived' as the target

First:
- input features: alone
- target: survived

Second:
- input features - embark_town
- target: survived

Third:
- input features -  age and family_size (embark_town)
- target: survived
- Justify your selection with reasoning.

## 3.1 Choose features and target

In [ ]:
# Separate features and target
X = df_encoded.drop("class", axis=1)
y = df_encoded["class"]

# Train a Decision Tree to get feature importances
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X, y)

# Get and sort feature importances
importances = pd.Series(dt.feature_importances_, index=X.columns).sort_values(ascending=False)

# Plot Top 10 Important Features
plt.figure(figsize=(10, 6))
sns.barplot(x=importances.values[:10], y=importances.index[:10])
plt.title("Top 10 Important Features (Decision Tree)")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.tight_layout()
plt.savefig('images/Top_10_features.png', dpi=300)  # Save the plot
plt.show()  # Optional: show in notebook
plt.close()

# Scaling – Required for distance-based models like SVM, MLP
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Wrap into DataFrame for easy use/inspection
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Confirm scaled data
print("\nScaled Feature Sample:")
print(X_scaled_df.head())


NameError: name 'df_encoded' is not defined

In [ ]:
# Create a new feature 'family_size' (sum of siblings/spouses and parents/children aboard)
df['family_size'] = df['sibsp'] + df['parch'] + 1  # +1 to include the passenger themselves

# Select relevant features for classification
features = ['alone', 'age', 'embark_town', 'sex', 'family_size']
target = 'survived'

# Extract relevant columns
titanic_classification = df[features + [target]]

# Encode categorical variable 'sex' (convert 'male'/'female' to 0/1)
titanic_classification['sex'] = titanic_classification['sex'].map({'male': 0, 'female': 1})

# Drop rows with missing values
titanic_classification = titanic_classification.dropna()

# Display the processed dataset
print(titanic_classification.head())

## 3.2 Define X (features) and y (target)
- Assign input features to X a pandas DataFrame with 1 or more input features
- Assign target variable to y (as applicable) - a pandas Series with a single target feature
- Again - use comments to run a single case at a time

- The follow starts with only the statements needed for case 1. 
- Double brackets [[ ]]  makes a 2D DataFrame
- Single brackets [ ]  make a 1D Series

In [ ]:
# Case 1 Assign input features to X = (alone)
X = df['alone']
# Assign target variable to y (as applicable)
y = df['survived']

In [ ]:
# Case 2 Assign input features to X = embarked
X = df['embark_town']
# Assign target variable to y (as applicable)   
y = df['survived']

In [ ]:
#  Case 3 Assign input features to X = 
X = df[['age', 'embark_town', 'family_size']]
# Assign target variable to y (as applicable)
y = df['survived']

### Reflection 3:

1. Why are these features selected? the features selected provide the most tell of survivability
2. Are there any features that are likely to be highly predictive of survival? Yes age and class

# Section 4. Train a Classification Model (Decision Tree)

## 4.1 Basic Train/Test split 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

print('Train size:', len(X_train))
print('Test size:', len(X_test))

## 4.2 Stratified Train/Test split

In [ ]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=123)

for train_indices, test_indices in splitter.split(X, y):
    train_set = X.iloc[train_indices]
    test_set = X.iloc[test_indices]

print('Train size:', len(train_set))
print('Test size:', len(test_set))

## 4.3 Compare Results


In [ ]:
# Compare results
print("Original Class Distribution:\n", y.value_counts(normalize=True))
print("Train Set Class Distribution:\n", train_set['pclass'].value_counts(normalize=True))
print("Test Set Class Distribution:\n", test_set['pclass'].value_counts(normalize=True))

### Reflection 4:

1. Why might stratification improve model performance? this ensures that the data is equallly representivate across the whole data set. 
2. How close are the training and test distributions to the original dataset?
3. Which split method produced better class balance?

## Section 5. Compare Alternative Models (SVC, NN)
In a Support Vector Machine, the kernel function defines how the algorithm transforms data to find a hyperplane that separates the classes. If the data is not linearly separable, changing the kernel can help the model find a better decision boundary.

SVC Kernel: Common Types

* RBF (Radial Basis Function) – Most commonly used; handles non-linear data well (default)
* Linear – Best for linearly separable data (straight line separation)
* Polynomial – Useful when the data follows a curved pattern
* Sigmoid – Similar to a neural network activation function; less common

_**Commenting the options in and out in the code can be helpful. The analyst decides which to use based on their understanding of the results.**_ 

In [ ]:
# RBF Kernel (default) - same as calling SVC()
svc_model = SVC(kernel='rbf')
svc_model.fit(X_train, y_train)

# Linear Kernel
svc_model = SVC(kernel='linear')
svc_model.fit(X_train, y_train)

# Polynomial Kernel (e.g., with degree=3)
svc_model = SVC(kernel='poly', degree=3)
svc_model.fit(X_train, y_train)

# Sigmoid Kernel
svc_model = SVC(kernel='sigmoid')
svc_model.fit(X_train, y_train)